# Saving properties to a CSV file
Python ships with a large library of handy modules, including the creation of [CSV files](https://en.wikipedia.org/wiki/Comma-separated_values), which is about the most simple structured file format possible and can easily opened in spreadsheet applications such as OpenOffice Calc or MS Excel.

To achieve this, we need to import [the CSV module](https://docs.python.org/2/library/csv.html) from the Python standard library, which allows us to read and write CSVs.

In [7]:
import ifcopenshell
m = ifcopenshell.open("data/hello_reiff_2021.ifc")
from utils.JupyterIFCRenderer import JupyterIFCRenderer
viewer = JupyterIFCRenderer(m, size=(400,300))
viewer

In [2]:
###########################################################################
# A simple script that iterates over all IfcPropertySets of the currently #
# selected object and writes them to a CSV file                           #
###########################################################################
import csv
selection = viewer.getSelectedProduct()
    #get the IfcProduct that is stored in the global variable 'selection'
with open(r'data/properties.csv', 'wb') as testfile:
    csv_writer = csv.writer(testfile, delimiter=';')
    for relDefinesByProperties in selection.IsDefinedBy:
         print("[{0}]".format(relDefinesByProperties.RelatingPropertyDefinition.Name))
         for prop in relDefinesByProperties.RelatingPropertyDefinition.HasProperties:
             print ("{:<20} :{}".format(prop.Name,prop.NominalValue.wrappedValue))
         print ("\n")
         csv_writer.writerow([selection.GlobalId, selection.Name, relDefinesByProperties.RelatingPropertyDefinition.Name,prop.Name,prop.NominalValue.wrappedValue])

[PSet_WallCommon]
Reference            :Wand Eins
AccousticRating      :54 dB
FireRating           :
Combustible          :True
SurfaceSpreadOfFlame :
ThermalTransmittance :3.14
IsExternal           :True
ExtendToStructure    :False
LoadBearing          :False
Compartmentation     :False




TypeError: a bytes-like object is required, not 'str'

                
# Reading properties from a file
Similarly to the above example to *export* information, we can quickly read information from external files (again, CSV to the rescue) and process it within our program.
We can e.g. write a simple model checker, to check the minimum heights of all doors etc.


## Minimalistic Model checker script
A simple to use external information defined in e.g. Excel to check values in an IFC file. Create a minimal CSV like:
```
  Element;Attribute;Min_height
  IfcDoor;OverallHeight;2.1
  IfcWindow;OverallHeight;1
```
An example is defined in the `../data` directory 


In [5]:
import csv
#open the file
with open(r'data/simple_check.csv') as checkfile:
    # create a reader, that reads the columns into a dictionary
    # using the first row as keys ("Element", "Attribute", "Min_height"
    reader = csv.DictReader(checkfile,delimiter=";")
    # iterate over all rows
    for row in reader:
        print(row['Element'], row["Attribute"],row['Min_height'])
        # get all elements 
        for element in m.by_type(row["Element"]):
            #print (f'{element} , {row["Attribute"]}')
            attribute_value=element.get_info()[row["Attribute"]]
            if float(attribute_value) < float(row['Min_height']):
                print (f"Element {element.GlobalId} violates the minimum height requirement with {attribute_value}")
     
    

FileNotFoundError: [Errno 2] No such file or directory: 'data/simple_check.csv'